# Test Adapters

> Adapter functions for the test suite, allowing easy instantiation and use of the BPE training and Tokenizer functionalities within test environments.

In [ ]:
#| default_exp adapters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Dict, Tuple, Optional

# Attempt to import from local modules, will be available after nbdev_export
try:
    from lm.bpe_training import train_bpe
    from lm.bpe_tokenizer_class import Tokenizer
except ImportError:
    # Fallback for direct notebook execution or if package not fully installed
    # This helps in iterative development but assumes a certain project structure
    # or that the user will run nbdev_export before tests that use these adapters.
    print("Warning: Could not import from .bpe_training or .bpe_tokenizer_class. \n"
          "Ensure these modules are exported (e.g., via `nbdev_export`) before using adapters extensively outside of tests run with nbdev/pytest.")
    # Define dummy versions if imports fail, so the notebook can still be processed by nbdev
    # However, actual tests requiring these will fail until real modules are available.
    def train_bpe(input_path: str, vocab_size: int, special_tokens: List[str]) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
        raise NotImplementedError("train_bpe is not available. Please ensure '02_bpe_training.ipynb' is exported.")
    
    class Tokenizer:
        def __init__(self, vocab: Dict[int, bytes], merges: List[Tuple[bytes, bytes]], special_tokens: Optional[List[str]] = None):
            raise NotImplementedError("Tokenizer class is not available. Please ensure '03_bpe_tokenizer_class.ipynb' is exported.")
        @classmethod
        def from_files(cls, vocab_filepath: str, merges_filepath: str, special_tokens: Optional[List[str]] = None) -> 'Tokenizer':
            raise NotImplementedError("Tokenizer.from_files is not available.")

In [ ]:
#| export
def run_train_bpe_adapter(
    input_path: str, 
    vocab_size: int, 
    special_tokens: List[str]
) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
    """Adapter for the `train_bpe` function.
    
    This function directly calls `train_bpe` with the provided arguments and returns its output.
    It's intended for use in test suites to easily invoke the BPE training process.

    Args:
        input_path (str): Path to the raw text file for training.
        vocab_size (int): Desired final vocabulary size.
        special_tokens (List[str]): List of special token strings.

    Returns:
        Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]: 
            The learned vocabulary and merge rules from `train_bpe`.
    """
    return train_bpe(input_path, vocab_size, special_tokens)

In [ ]:
#| export
def get_tokenizer_from_files_adapter(
    vocab_path: str, 
    merges_path: str, 
    special_tokens: Optional[List[str]] = None
) -> Tokenizer:
    """Adapter for the `Tokenizer.from_files` classmethod.

    This function calls `Tokenizer.from_files` to load a tokenizer instance
    from serialized vocabulary and merge files.

    Args:
        vocab_path (str): Path to the JSON file containing the vocabulary.
        merges_path (str): Path to the JSON file containing the merge rules.
        special_tokens (Optional[List[str]], optional): List of special token strings.
            Defaults to None.

    Returns:
        Tokenizer: An instance of the Tokenizer class.
    """
    return Tokenizer.from_files(vocab_path, merges_path, special_tokens)

In [ ]:
#| export
def get_tokenizer_adapter(
    vocab: Dict[int, bytes], 
    merges: List[Tuple[bytes, bytes]], 
    special_tokens: Optional[List[str]] = None
) -> Tokenizer:
    """Adapter for the `Tokenizer` constructor.

    This function directly instantiates the `Tokenizer` class with the provided
    vocabulary, merge rules, and special tokens.

    Args:
        vocab (Dict[int, bytes]): Vocabulary mapping token IDs to byte sequences.
        merges (List[Tuple[bytes, bytes]]): List of BPE merge rules.
        special_tokens (Optional[List[str]], optional): List of special token strings.
            Defaults to None.

    Returns:
        Tokenizer: An instance of the Tokenizer class.
    """
    return Tokenizer(vocab, merges, special_tokens)

In [ ]:
#| hide
# Example Usage (for illustration, actual testing would be in separate test files)
if __name__ == '__main__':
    import tempfile
    import os
    from lm.utils import save_tokenizer_components # Assuming utils are exported

    print("Testing adapter functions (requires exported modules and dummy data)...")

    # Dummy data for testing adapters
    dummy_vocab = {i: bytes([i]) for i in range(256)}
    dummy_vocab[256] = b"<EOS>"
    dummy_merges = [(b'e', b's')] 
    dummy_special_tokens = ["<EOS>"]

    # Test get_tokenizer_adapter
    try:
        tokenizer_instance = get_tokenizer_adapter(dummy_vocab, dummy_merges, dummy_special_tokens)
        assert isinstance(tokenizer_instance, Tokenizer), "get_tokenizer_adapter failed to return Tokenizer instance"
        print("get_tokenizer_adapter: PASSED")
    except Exception as e:
        print(f"get_tokenizer_adapter: FAILED - {e}")

    # Setup for file-based adapters
    temp_dir = tempfile.mkdtemp()
    vocab_file = os.path.join(temp_dir, "dummy_vocab.json")
    merges_file = os.path.join(temp_dir, "dummy_merges.json")

    try:
        save_tokenizer_components(dummy_vocab, dummy_merges, vocab_file, merges_file)
        
        # Test get_tokenizer_from_files_adapter
        tokenizer_from_files = get_tokenizer_from_files_adapter(vocab_file, merges_file, dummy_special_tokens)
        assert isinstance(tokenizer_from_files, Tokenizer), "get_tokenizer_from_files_adapter failed"
        print("get_tokenizer_from_files_adapter: PASSED")

        # Test run_train_bpe_adapter (basic call, requires a dummy input file)
        dummy_input_text_file = os.path.join(temp_dir, "dummy_input.txt")
        with open(dummy_input_text_file, "w", encoding="utf-8") as f:
            f.write("test text for bpe training est est.")
        
        trained_vocab, trained_merges = run_train_bpe_adapter(dummy_input_text_file, 260, ["<EOS>"])
        assert isinstance(trained_vocab, dict) and isinstance(trained_merges, list), "run_train_bpe_adapter failed"
        print(f"run_train_bpe_adapter: PASSED (vocab_size={len(trained_vocab)}, merges={len(trained_merges)})")

    except NotImplementedError as e:
        print(f"Adapter tests skipped due to missing modules: {e}")
    except Exception as e:
        print(f"Adapter tests FAILED during file operations: {e}")
    finally:
        # Cleanup temporary directory and files
        if os.path.exists(dummy_input_text_file):
            os.remove(dummy_input_text_file)
        if os.path.exists(vocab_file):
            os.remove(vocab_file)
        if os.path.exists(merges_file):
            os.remove(merges_file)
        if os.path.exists(temp_dir):
            os.rmdir(temp_dir)
    print("Adapter function tests completed.")
